In [ ]:
import torch
import numpy as np
from torch.autograd import Variable

torch.manual_seed(783)

x_train = Variable(torch.from_numpy(x_train_np)).float()
y_train = Variable(torch.from_numpy(y_train_np)).float()

In [ ]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.lstm = torch.nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=data_width,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = torch.nn.Linear(64, label_team_width)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.lstm(x, None)   # None represents zero initial hidden state
    
        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out


rnn = RNN()
print(rnn)
rnn.cuda()

batch_size = 100 # time sequence
optimizer = torch.optim.Adam(rnn.parameters())   # optimize all cnn parameters
loss_func = torch.nn.CrossEntropyLoss()          # the target label is not one-hotted
loss_func = loss_func.cuda()

# training and testing
for epoch in range(100):
    for idx in range(int(6000/batch_size)):

        print('idx: ', idx*batch_size)
        x_train_ = x_train[:, idx*batch_size : (idx+1)*batch_size, :]
        y_train_ = y_train[:, idx*batch_size : (idx+1)*batch_size, :]

        x_train_batch = x_train_.cuda()
        y_train_batch = y_train_.cuda()

        y_pred_train = rnn(x_train_batch)
        loss_train = loss_func(y_pred_train, y_train_batch.type(torch.cuda.LongTensor))
    
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        print(loss_train.cpu().detach().numpy())

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(num_features=16),
            torch.nn.MaxPool2d(kernel_size=4, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(num_features=24),
            torch.nn.MaxPool2d(kernel_size=4, stride=2)
        )
        self.conv = torch.nn.Conv2d(in_channels=24, out_channels=2, kernel_size=3, stride=1, padding=2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.conv(x)
        return x